In [38]:
from datasets import load_dataset, disable_caching
disable_caching()
from gpn.data import GenomeMSA, Tokenizer, ReverseComplementer
import gpn.model
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy.stats import pearsonr, spearmanr
import seaborn as sns
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
import tempfile
from transformers import AutoModel, Trainer, TrainingArguments
import torch
import torch.nn.functional as F
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import yaml

In [2]:
with open("../../config/config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

In [4]:
dataset = load_dataset("gonzalobenegas/gwas", split="test")
V = dataset.to_pandas()
V

,chrom,pos,ref,alt,pip,maf,trait,label,consequence,tss_dist,exon_dist,match_group
0,1,930312,C,G,0.001963,0.000683,,False,missense_variant,0,0,missense_variant_5
1,1,976215,A,G,0.005418,0.199367,,False,missense_variant,2703,0,missense_variant_39
2,1,1203822,T,C,0.006379,0.059558,,False,synonymous_variant,1856,0,synonymous_variant_0
3,1,1224844,C,T,0.001119,0.096291,,False,intron_variant,3750,876,intron_variant_11
4,1,1291417,C,G,0.003143,0.003690,,False,missense_variant,4308,0,missense_variant_40
...,...,...,...,...,...,...,...,...,...,...,...,...
4589,22,50190508,G,A,1.000000,0.073316,Plt,True,intron_variant,88,88,intron_variant_709
4590,22,50309599,G,A,0.002612,0.005661,,False,3_prime_UTR_variant,1953,0,3_prime_UTR_variant_61
4591,22,50395752,A,G,0.001928,0.083293,,False,intron_variant,23680,1617,intron_variant_692
4592,22,50516206,G,A,0.978163,0.095820,MCH,True,intron_variant,6317,239,intron_variant_710


In [5]:
genome_msa = GenomeMSA(
    config["gpn_msa"]["msa_path"], subset_chroms=dataset.unique("chrom"), in_memory=False
)
genome_msa

Loading MSA...
Loading MSA... Done


In [9]:
class VEPEmbedding(torch.nn.Module):
    def __init__(self, model_path):
        super().__init__()
        self.model = AutoModel.from_pretrained(model_path)

    def get_embedding(self, input_ids, aux_features):
        return self.model(
            input_ids=input_ids, aux_features=aux_features,
        ).last_hidden_state

    def get_scores(self, input_ids_ref, aux_features_ref, input_ids_alt, aux_features_alt):
        embedding_ref = self.get_embedding(input_ids_ref, aux_features_ref)
        embedding_alt = self.get_embedding(input_ids_alt, aux_features_alt)
        return embedding_ref, embedding_alt #torch.stack((embedding_ref, embedding_alt))

    def forward(
        self,
        input_ids_ref_fwd=None,
        aux_features_ref_fwd=None,
        input_ids_alt_fwd=None,
        aux_features_alt_fwd=None,
        input_ids_ref_rev=None,
        aux_features_ref_rev=None,
        input_ids_alt_rev=None,
        aux_features_alt_rev=None,
    ):
        fwd = self.get_scores(
            input_ids_ref_fwd, aux_features_ref_fwd, input_ids_alt_fwd, aux_features_alt_fwd,
        )
        return fwd
        #rev = self.get_scores(
        #    input_ids_ref_rev, aux_features_ref_rev, input_ids_alt_rev, aux_features_alt_rev,
        #)
        #return (fwd + rev) / 2


class VEPEmbeddingInference(object):
    def __init__(self, model_path, genome_msa, window_size):
        self.model = VEPEmbedding(model_path)
        self.genome_msa = genome_msa
        self.window_size = window_size
        self.tokenizer = Tokenizer()
        self.reverse_complementer = ReverseComplementer()

    def tokenize_function(self, V):
        # we convert from 1-based coordinate (standard in VCF) to
        # 0-based, to use with GenomeMSA
        chrom = np.array(V["chrom"])
        pos = np.array(V["pos"]) - 1
        start = pos - self.window_size // 2
        end = pos + self.window_size // 2
        
        msa_fwd, msa_rev = self.genome_msa.get_msa_batch_fwd_rev(
            chrom,
            start,
            end,
            tokenize=True,
        )
        pos_fwd = self.window_size // 2
        pos_rev = pos_fwd - 1 if self.window_size % 2 == 0 else pos_fwd

        ref_fwd = np.array(
            [np.frombuffer(x.encode("ascii"), dtype="S1") for x in V["ref"]]
        )
        alt_fwd = np.array(
            [np.frombuffer(x.encode("ascii"), dtype="S1") for x in V["alt"]]
        )
        ref_rev = self.reverse_complementer(ref_fwd)
        alt_rev = self.reverse_complementer(alt_fwd)

        def prepare_output(msa, pos, ref, alt):
            ref, alt = self.tokenizer(ref.flatten()), self.tokenizer(alt.flatten())
            input_ids, aux_features = msa[:, :, 0], msa[:, :, 1:]
            assert (
                input_ids[:, pos] == ref
            ).all(), f"{input_ids[:, pos].tolist()}, {ref.tolist()}"
            input_ids_alt = input_ids.copy()
            input_ids_alt[:, pos] = alt
            input_ids = input_ids.astype(np.int64)
            input_ids_alt = input_ids_alt.astype(np.int64)
            return input_ids, aux_features, input_ids_alt, aux_features

        res = {}
        (
            res["input_ids_ref_fwd"],
            res["aux_features_ref_fwd"],
            res["input_ids_alt_fwd"],
            res["aux_features_alt_fwd"],
        ) = prepare_output(msa_fwd, pos_fwd, ref_fwd, alt_fwd)
        (
            res["input_ids_ref_rev"],
            res["aux_features_ref_rev"],
            res["input_ids_alt_rev"],
            res["aux_features_alt_rev"],
        ) = prepare_output(msa_rev, pos_rev, ref_rev, alt_rev)
        return res

    def postprocess(self, pred):
        return pred

In [10]:
inference = VEPEmbeddingInference(
    config["gpn_msa"]["model_path"],
    genome_msa,
    config["gpn_msa"]["window_size"],
)

In [11]:
dataset.set_transform(inference.tokenize_function)
training_args = TrainingArguments(
    output_dir=tempfile.TemporaryDirectory().name,
    per_device_eval_batch_size=2048,
    dataloader_num_workers=8,
    remove_unused_columns=False,
    torch_compile=True,
    fp16=True,
)
trainer = Trainer(model=inference.model, args=training_args)
pred = trainer.predict(test_dataset=dataset).predictions
res = inference.postprocess(pred)

In [12]:
pred[0].shape, pred[1].shape

((4594, 128, 768), (4594, 128, 768))

In [13]:
full_embed_ref, full_embed_alt = pred

In [14]:
embed_ref = full_embed_ref.mean(axis=1)
embed_alt = full_embed_alt.mean(axis=1)
embed_ref.shape, embed_alt.shape

((4594, 768), (4594, 768))

In [26]:
cosine_dist = 1 - F.cosine_similarity(torch.tensor(embed_ref), torch.tensor(embed_alt)).numpy()
cosine_dist.shape

(4594,)

In [27]:
euclidean_dist = F.pairwise_distance(torch.tensor(embed_ref), torch.tensor(embed_alt)).numpy()
euclidean_dist.shape

(4594,)

In [30]:
minus_inner_product = -(embed_ref*embed_alt).sum(axis=1)
minus_inner_product.shape

(4594,)

In [28]:
roc_auc_score(V.label, cosine_dist)

0.5670621652781382

In [29]:
roc_auc_score(V.label, euclidean_dist)

0.5425306313680902

In [31]:
roc_auc_score(V.label, minus_inner_product)

0.5882266036087653

In [34]:
np.hstack((embed_ref, embed_alt)).shape

(4594, 1536)

In [68]:
all_features = [
    np.hstack((embed_ref, embed_alt)), # 0.66
    embed_ref,                         # 0.66
    embed_alt,                         # 0.66
    embed_ref - embed_alt,             # 0.57
    np.abs(embed_ref-embed_alt),       # 0.62
    embed_ref * embed_alt,             # 0.65
]

In [74]:
def train_predict_logistic_regression(X_train, y_train, X_test):
    clf = Pipeline([
        ('scaler', RobustScaler()),
        ('linear', LogisticRegressionCV(
            class_weight="balanced",
            scoring="roc_auc",
            Cs=np.logspace(-10, 0, 20),
            random_state=42,
            n_jobs=-1,
        ))
    ])
    clf.fit(X_train, y_train)
    print(clf.named_steps["linear"].C_)
    return clf.predict_proba(X_test)[:, 1]

ODD_EVEN_CHROMS = [
    [str(i) for i in range(1, 23, 2)] + ['X'],
    [str(i) for i in range(2, 23, 2)] + ['Y'],
]
def train_predict_loop(features):
    for chroms in tqdm(ODD_EVEN_CHROMS):
        mask_train = V.chrom.isin(chroms)
        mask_test = ~mask_train
        V.loc[mask_test, "score"] = train_predict_logistic_regression(
            features[mask_train], V[mask_train].label, features[mask_test]
        )
    return V.score.values.copy()

scores = [train_predict_loop(features) for features in all_features]

  0%|          | 0/2 [00:00<?, ?it/s]/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    

[0.00233572]


/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mod

[0.00233572]


  0%|          | 0/2 [00:00<?, ?it/s]/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

...

[0.00069519]


  0%|          | 0/2 [00:00<?, ?it/s]/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    

[0.00020691]


/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

[0.00069519]


In [76]:
for score in scores:
    print(f"{roc_auc_score(V.label, score):.2f}")

0.66
0.66
0.66
0.57
0.62
0.65
